# Recognizing hand-written digits
This notebook demonstrates how to use a convolutional neural network for image recognition.
We're using the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). 

You need a specially prepared dataset for this notebook. You can prepare the dataset for this notebook using the notebook "Prepare the dataset.ipynb". 

## The network structure
The neural network has two convolution layers and two max-pooling layers mixed together.
The output of the neural network is a softmax activated dense layer with 10 neurons. Each for one of the different labels that we can predict with this model.

In [1]:
from cntk.layers import Convolution2D, Sequential, Dense, MaxPooling
from cntk.ops import log_softmax, relu, sigmoid
from cntk.initializer import glorot_uniform
from cntk import input_variable, default_options

In [2]:
features = input_variable((3,28,28))
labels = input_variable(10)

In [3]:
with default_options(initialization=glorot_uniform, activation=relu):
    model = Sequential([
        Convolution2D(filter_shape=(5,5), strides=(1,1), num_filters=8, pad=True),
        MaxPooling(filter_shape=(2,2), strides=(2,2)),
        Convolution2D(filter_shape=(5,5), strides=(1,1), num_filters=16, pad=True),
        MaxPooling(filter_shape=(3,3), strides=(3,3)),
        Dense(10, activation=log_softmax)
    ])

In [4]:
z = model(features)

The loss for the model is a categorical cross entropy. We're using a `Function` object to combine the loss with a metric to measure the performance of the model. This `criterion_factory` is used to create the objective for the training logic. We're using a SGD learner for this model.

In [5]:
from cntk import Function
from cntk.losses import cross_entropy_with_softmax
from cntk.metrics import classification_error

@Function
def criterion_factory(output, targets):
    loss = cross_entropy_with_softmax(output, targets)
    metric = classification_error(output, targets)
    
    return loss, metric

In [6]:
loss = criterion_factory(z, labels)

In [7]:
from cntk.learners import sgd

learner = sgd(z.parameters, lr=0.2)

## The data source
The data is stored as images on disk with a mapping file that combines the filename of the images with the label for each of the images. We're using random transforms during training to augment the training data in an attempt to improve performance.

Note that these transforms are not applied during testing. 

In [8]:
import os
from cntk.io import MinibatchSource, StreamDef, StreamDefs, ImageDeserializer, INFINITELY_REPEAT
import cntk.io.transforms as xforms

def create_datasource(folder, train=True, max_sweeps=INFINITELY_REPEAT):
    mapping_file = os.path.join(folder, 'mapping.bin')
    
    image_transforms = []
    
    if train:
        image_transforms += [
            xforms.crop(crop_type='randomside', side_ratio=0.8)
        ]
        
    # Always scale the image, the cropping breaks them during training.
    # The scaling fixes the problem.
    image_transforms += [
        xforms.scale(width=28, height=28, channels=3, interpolations='linear')
    ]
    
    stream_definitions = StreamDefs(
        features=StreamDef(field='image', transforms=image_transforms),
        labels=StreamDef(field='label', shape=10)
    )
    
    deserializer = ImageDeserializer(mapping_file, stream_definitions)
    
    return MinibatchSource(deserializer, max_sweeps=max_sweeps)

Training data and testing data is stored in separate folders.
You need a separate data source for both.

In [9]:
train_datasource = create_datasource('mnist_train')
test_datasource = create_datasource('mnist_test', max_sweeps=1, train=False)

## Training the model
The model is trained for one epoch with a batchsize of 64. We've added the progress printer to visualize the output of the training session. We've also included the test set here to validate the performance of the model.

In [10]:
from cntk.logging import ProgressPrinter
from cntk.train import TestConfig


progress_writer = ProgressPrinter(0)

test_config = TestConfig(test_datasource)

input_map = {
    features: train_datasource.streams.features,
    labels: train_datasource.streams.labels
}

loss.train(train_datasource, 
           max_epochs=1,
           minibatch_size=64,
           epoch_size=60000, 
           parameter_learners=[learner], 
           model_inputs_to_streams=input_map,  
           callbacks=[progress_writer, test_config])

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.2
      165        165      0.859      0.859            64
 1.96e+07   2.94e+07      0.875      0.883           192
  8.4e+06          4      0.886      0.895           448
 3.92e+06          4      0.893      0.898           960
  1.9e+06          4      0.893      0.894          1984
 9.35e+05    1.4e+03      0.895      0.896          4032
 4.64e+05          4      0.894      0.893          8128
 2.31e+05          4      0.892       0.89         16320
 1.15e+05          4      0.891       0.89         32704
Finished Evaluation [1]: Minibatch[1-313]: metric = 88.65% * 10000;


{'updates': [{'loss': 164.5167694091797, 'metric': 0.859375, 'samples': 64},
  {'loss': 29417017.741615295, 'metric': 0.8828125, 'samples': 128},
  {'loss': 4.0, 'metric': 0.89453125, 'samples': 256},
  {'loss': 4.0, 'metric': 0.8984375, 'samples': 512},
  {'loss': 4.0, 'metric': 0.8935546875, 'samples': 1024},
  {'loss': 1402.875, 'metric': 0.89599609375, 'samples': 2048},
  {'loss': 4.0, 'metric': 0.89306640625, 'samples': 4096},
  {'loss': 4.0, 'metric': 0.89013671875, 'samples': 8192},
  {'loss': 4.0, 'metric': 0.88983154296875, 'samples': 16384}],
 'epoch_summaries': [{'loss': 62808.21333333333,
   'metric': 0.89035,
   'samples': 60000}],
 'test_summary': {'metric': 0.8865, 'samples': 10000}}